## Capstone - Data Cleansing

In [1]:
import pandas as pd
import numpy as np
import re
import string

In [2]:
# Load in data 
df = pd.read_csv('final_data.csv')
df.head(2)

,Unnamed: 0,index,country_name,country_code,tree_cover,tree_cover_%_2000,tree_cover_loss_2014,cover_gain,forest_type_nat,forest_type_pri,...,RAMSAR,WORLD HERITAGE,NLBI,ILO 169,Country,Agricultural land (% of land area) 2013,Population change 2015 %,GDP per capita (current US$) 2015,GDP growth (annual %) 2015,Food production index 2013
0,0,0,Afghanistan,AFG,205KHa,0.0,2.0,303.0,NaN,NaN,...,Non Party,Ratification,Member State,Not Ratified,Afghanistan,58.1,2.8,590.3,1.5,119.8
1,1,2,Albania,ALB,675.8KHa,NaN,NaN,NaN,0.77,0.11,...,Contracting Party,Ratification,Member State,Not Ratified,Albania,43.3,-0.2,"3,965.00",2.6,127.9


In [3]:
# Drop rows where data was unavailable for Global Forest Watch

df.dropna(axis =0, how = 'all', subset=[u'tree_cover', u'tree_cover_%_2000',
       u'tree_cover_loss_2014', u'cover_gain', u'forest_type_nat',
       u'forest_type_pri', u'forest_type_pla', u'loss_outside_plant',
       u'loss_outside_plant_%', u'certified_forest', u'total_economy',
       u'percentage_economy', u'employment', u'reforestation_rate_10',
       u'carbon', u'GHG_emissions'], inplace = True)

In [4]:
# Drop redundent columns

df.drop(['Unnamed: 0', 'index', 'index.1', 'Country', 'loss_outside_plant',
        'loss_outside_plant_%'], axis = 1, inplace=True)

In [5]:
# If tree cover information was unavailable for a country then we would like to drop them

df.dropna(axis = 0, subset=['tree_cover'], inplace = True)

In [6]:
df.shape


(174, 31)

#### Changing data types

A number of columns are strings containing digits. We would like to extract only the numerical information.

In [7]:
# Functions 

# Function that standardises tree cover into hectors 
def convert(string):
    if 'KHa' in string:
        return int(re.search('\d+', string).group()) * 1000
    elif 'MHa' in string:
        return int(re.search('\d+', string).group()) * 1000000
    else:
        return int(re.search('\d+', string).group())
    
# Functions that gets numbers from text

def get_decimal(string):
    try:
        return float(re.search('\d+.\d+', string).group())
    except:
        return np.nan
    
def get_integer(string):
    try:
        return int(re.search('\d+', string).group())
    except:
        return np.nan

In [8]:
# Apply functions to columns

df['tree_cover'] = df['tree_cover'].apply(convert)
df['total_economy'] = df['total_economy'].apply(get_decimal)
df['percentage_economy'] = df['percentage_economy'].apply(get_decimal)
df['employment'] = df['employment'].apply(get_integer)
df['reforestation_rate_10'] = df['reforestation_rate_10'].apply(get_integer)
df['carbon'] = df['carbon'].str.replace(',', '')
df['carbon'] = df['carbon'].apply(get_integer)
df['GHG_emissions'] = df['GHG_emissions'].apply(get_decimal)
df['certified_forest'] = df['certified_forest'].apply(get_integer)

In [9]:
df['GDP per capita (current US$) 2015'] = df['GDP per capita (current US$) 2015'].str.replace(',','')
df['GDP per capita (current US$) 2015'] = df['GDP per capita (current US$) 2015'].astype(float)

In [10]:
# Creating additional columns

# Normalising forest loss which will eventually become our target
df['loss_14_%'] = (df['tree_cover_loss_2014'] / df['tree_cover']) * 100

# Percentage of certified forest
df['certified_%'] = (df['certified_forest'] / df['tree_cover']) * 100

# Percentage of forest gained
df['cover_gained_%'] = (df['cover_gain'] / df['tree_cover']) * 100

# Forest loss to quartiles for EDA
df['loss_quartile'] = pd.qcut(df['loss_14_%'], 4, labels = [1 ,2 ,3 ,4])

In [11]:
# Cleaning convention columns

# Cleaning categories
dicty = {'Non-Party': 'Non Party', 'Withdrawn': 'Non Party', '????': 'Non Party', '-': 'Not Ratified',
         'Non Member': 'Non Party', 'Not Ratified': 'Non Party'  }
df = df.replace(dicty)

# Dummify UNFCC columns
df = pd.get_dummies(df, columns = ['UNFCCC'], drop_first= False)

# Membership
members = {'Ratification': 1, 'Accession': 1, 'Non Party': 0, 'Approval': 1, 'Acceptance': 1,
          'Succession' : 1, 'Signature': 1, 'Consuming Member': 1, 'Producing Member': 1, 'Continuation': 1,
          'Contracting Party': 1, 'Member State': 1, 'Non Member State': 0 }
df.replace(members, inplace = True)

In [12]:
# Save to csv

df.to_csv('cleaned_data.csv')

In [15]:
# Import data to Local Postgres Database

import sqlalchemy
from sqlalchemy import create_engine

engine = create_engine('postgresql://jayecribb@localhost:5432/capstone')
df.to_sql('cleaned_data',  engine, if_exists='replace')